In [1]:
import pandas as pd
import numpy as np
import gmaps
import requests
from citipy import citipy
from config import g_key
from datetime import datetime
import time
from scipy.stats import linregress

In [2]:
city_data=pd.read_csv('WeatherPy_challenge.csv')
city_data.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Aras,ES,2020-03-01 16:40:41,42.56,-2.36,64.40,34,98,13.87,overcast clouds,0.00,0.00
1,1,Butaritari,KI,2020-03-01 16:40:41,3.07,172.79,83.16,80,69,21.00,light rain,0.19,0.00
2,2,Port Alfred,ZA,2020-03-01 16:40:42,-33.59,26.89,69.01,84,0,5.01,clear sky,0.00,0.00
3,3,Naryan-Mar,RU,2020-03-01 16:40:42,67.67,53.09,15.57,93,100,16.78,light snow,0.00,0.31
4,4,Umm Lajj,SA,2020-03-01 16:40:42,25.02,37.27,64.71,57,0,11.90,clear sky,0.00,0.00


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input('What is the maximum temperature you would like for your trip? '))
raining=input('Do you want it to be raining? (yes/no) ')
snowing=input('Do you want it to be snowing? (yes/no) ')

preferred_cities=city_data.loc[(city_data['Max Temp']<=max_temp)& \
                               (city_data['Max Temp']>= min_temp)]

# 4 different Combos:
if raining == 'no' and snowing == 'no':
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']==0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']==0)]
    
elif raining == 'no' and snowing == 'yes':
    preferred_citie=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']>0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']==0)]
    
elif raining == 'yes' and snowing == 'no':
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']==0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']>0)]

else:
    preferred_cities=preferred_cities.loc[(preferred_cities['Snow inches (last 3 hrs)']>0)&  
                                          (preferred_cities['Rain inches (last 3 hrs)']>0)]
preferred_cities.head(10)

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,2,Port Alfred,ZA,2020-03-01 16:40:42,-33.59,26.89,69.01,84,0,5.01,clear sky,0.0,0.0
5,5,Georgetown,MY,2020-03-01 16:37:54,5.41,100.34,84.20,88,20,1.95,few clouds,0.0,0.0
8,8,Hermanus,ZA,2020-03-01 16:40:43,-34.42,19.23,73.99,73,50,11.01,scattered clouds,0.0,0.0
10,10,Canico,PT,2020-03-01 16:40:43,32.63,-16.85,69.01,77,40,3.36,scattered clouds,0.0,0.0
15,15,Labuhan,ID,2020-03-01 16:40:44,-6.88,112.21,75.16,94,100,3.15,overcast clouds,0.0,0.0
16,16,Cayenne,GF,2020-03-01 16:40:45,4.93,-52.33,84.20,70,75,10.29,broken clouds,0.0,0.0
17,17,Mahebourg,MU,2020-03-01 16:36:39,-20.41,57.70,81.00,88,40,2.24,scattered clouds,0.0,0.0
19,19,Saldanha,ZA,2020-03-01 16:40:45,-33.01,17.94,78.80,39,5,5.82,clear sky,0.0,0.0
25,25,Airai,TL,2020-03-01 16:40:47,-8.93,125.41,68.07,83,100,5.50,overcast clouds,0.0,0.0
28,28,Meulaboh,ID,2020-03-01 16:40:48,4.14,96.13,85.50,69,75,2.68,broken clouds,0.0,0.0


In [5]:
hotel_vacation=preferred_cities[['City','Country','Max Temp','Lat','Lng','Current Description']].copy()
hotel_vacation['Hotel Name']=''
hotel_vacation

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Port Alfred,ZA,69.01,-33.59,26.89,clear sky,
5,Georgetown,MY,84.20,5.41,100.34,few clouds,
8,Hermanus,ZA,73.99,-34.42,19.23,scattered clouds,
10,Canico,PT,69.01,32.63,-16.85,scattered clouds,
15,Labuhan,ID,75.16,-6.88,112.21,overcast clouds,
...,...,...,...,...,...,...,...
527,Vung Tau,VN,78.31,10.35,107.08,overcast clouds,
542,Broken Hill,AU,73.40,-31.95,141.43,overcast clouds,
543,Port Moresby,PG,77.00,-9.44,147.18,scattered clouds,
549,Ko Samui,TH,80.82,9.54,99.94,scattered clouds,


In [6]:
params={
    'radius':5000,
    'types':'lodging',
    'key':g_key}
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotels = requests.get(base_url, params=params).json()

In [7]:
for index, row in hotel.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_vacation.loc[index,"Hotel Name"]=hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

NameError: name 'hotel' is not defined

In [8]:

hotel_vacation

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Port Alfred,ZA,69.01,-33.59,26.89,clear sky,
5,Georgetown,MY,84.20,5.41,100.34,few clouds,
8,Hermanus,ZA,73.99,-34.42,19.23,scattered clouds,
10,Canico,PT,69.01,32.63,-16.85,scattered clouds,
15,Labuhan,ID,75.16,-6.88,112.21,overcast clouds,
...,...,...,...,...,...,...,...
527,Vung Tau,VN,78.31,10.35,107.08,overcast clouds,
542,Broken Hill,AU,73.40,-31.95,141.43,overcast clouds,
543,Port Moresby,PG,77.00,-9.44,147.18,scattered clouds,
549,Ko Samui,TH,80.82,9.54,99.94,scattered clouds,


In [9]:
output_data_file='WeatherPy_vacation.csv'
hotel_vacation.to_csv(output_data_file, index_label='City_ID')

In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]

NameError: name 'hotel_df' is not defined

In [11]:
locations=hotel_df[['Lat','Lng']]
max_temp=hotel_df['Max Temp']
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

NameError: name 'hotel_df' is not defined

In [12]:
conda install -c conda-forge ipywidgets


Note: you may need to restart the kernel to use updated packages.


'conda' is not recognized as an internal or external command,
operable program or batch file.
